In [1]:
# !pip install biopython
import Bio
from Bio import SeqIO
import torch

train_seq = list(SeqIO.parse("./data/train.fasta", "fasta"))
# print(train_seq[1].id)
# print(train_seq[1].seq)
print(train_seq[2221])

class_indexs = ["nonarg","beta_lactam","bacitracin","polymyxin","fosfomycin","tetracycline","aminoglycoside","macrolide-lincosamide-streptogramin",
                "multidrug","chloramphenicol","glycopeptide","sulfonamide","quinolone","trimethoprim","rifampin"] # all the antibiotic classes 15

codes = ['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L',
         'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y'] # 20 amino acids for one hot encoding

def create_dict(codes):
  char_dict = {}
  for index, val in enumerate(codes):
    char_dict[val] = index+1

  return char_dict

char_dict = create_dict(codes)

print(char_dict)


# for i in train_seq:
#     valid = True
#     if 'argannot' in i.description or 'deepARG' in i.description:
#         for j in class_indexs:
#             if j in i.description:
#                 valid = False
#                 break
#         if valid:
#             print(i.description)
#             print(i.seq)
#             print(len(i.seq))
#             print(len(i.description))
#             print("-----------------------------------------------------")
                


print(len(class_indexs))

ID: AAC75429.1|FEATURES|CARD|multidrug|evgS|antibiotic
Name: AAC75429.1|FEATURES|CARD|multidrug|evgS|antibiotic
Description: AAC75429.1|FEATURES|CARD|multidrug|evgS|antibiotic efflux|0
Number of features: 0
Seq('MKFLPYIFLLCCGLWSTISFADEDYIEYRGISSNNRVTLDPLRLSNKELRWLAS...KND')
{'A': 1, 'C': 2, 'D': 3, 'E': 4, 'F': 5, 'G': 6, 'H': 7, 'I': 8, 'K': 9, 'L': 10, 'M': 11, 'N': 12, 'P': 13, 'Q': 14, 'R': 15, 'S': 16, 'T': 17, 'V': 18, 'W': 19, 'Y': 20}
15


In [1]:
!pip install evaluate

  Obtaining dependency information for evaluate from https://files.pythonhosted.org/packages/70/63/7644a1eb7b0297e585a6adec98ed9e575309bb973c33b394dae66bc35c69/evaluate-0.4.1-py3-none-any.whl.metadata
  Obtaining dependency information for responses<0.19 from https://files.pythonhosted.org/packages/79/f3/2b3a6dc5986303b3dd1bbbcf482022acb2583c428cd23f0b6d37b1a1a519/responses-0.18.0-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/84.1 kB ? eta -:--:--
   ---------------------------------------- 84.1/84.1 kB 4.6 MB/s eta 0:00:00


In [1]:
import torch
import torch.nn as nn
import re
import numpy as np
import os
import requests
from tqdm import tqdm
from transformers import AutoTokenizer, EsmForSequenceClassification, AutoModelForSequenceClassification
# set gpu
device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"
device

from Bio import SeqIO
arg_dict = {'aminoglycoside': 0, 'macrolide-lincosamide-streptogramin': 1, 'polymyxin': 2,
'fosfomycin': 3, 'trimethoprim': 4, 'bacitracin': 5, 'quinolone': 6, 'multidrug': 7,
'chloramphenicol': 8, 'tetracycline': 9, 'rifampin': 10, 'beta_lactam': 11,
'sulfonamide': 12, 'glycopeptide': 13, 'nonarg': 14}

model = AutoModelForSequenceClassification.from_pretrained("facebook/esm2_t6_8M_UR50D", num_labels=15)
model = model.to(device)
print(model)





Some weights of EsmForSequenceClassification were not initialized from the model checkpoint at facebook/esm2_t6_8M_UR50D and are newly initialized: ['classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight', 'classifier.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


EsmForSequenceClassification(
  (esm): EsmModel(
    (embeddings): EsmEmbeddings(
      (word_embeddings): Embedding(33, 320, padding_idx=1)
      (dropout): Dropout(p=0.0, inplace=False)
      (position_embeddings): Embedding(1026, 320, padding_idx=1)
    )
    (encoder): EsmEncoder(
      (layer): ModuleList(
        (0-5): 6 x EsmLayer(
          (attention): EsmAttention(
            (self): EsmSelfAttention(
              (query): Linear(in_features=320, out_features=320, bias=True)
              (key): Linear(in_features=320, out_features=320, bias=True)
              (value): Linear(in_features=320, out_features=320, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
              (rotary_embeddings): RotaryEmbedding()
            )
            (output): EsmSelfOutput(
              (dense): Linear(in_features=320, out_features=320, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
            (LayerNorm): LayerNorm((320,), eps=1e-05,

In [2]:
# data test
model.eval()
class SeqDataset(torch.utils.data.Dataset):
    
    def __init__(self, data_root, data_label, attention_mask):
        self.data = data_root
        self.label = data_label
        self.attention_mask = attention_mask
        
    def __getitem__(self, index):
        dict = {}
        dict["labels"] = self.label[index]
        dict["input_ids"] = self.data[index]
        dict["attention_mask"] = self.attention_mask[index]
        return dict
    
    def __len__(self):
        return len(self.data)

train_seq = []
label = []
test_seq = []
test_name = []

val_seq = []
val_label = []

for seq_record in SeqIO.parse("./data/train.fasta", "fasta"):
    if "sp" in seq_record.id:
        label.append(14)
    else:
        id_list = seq_record.id.split("|")
        label.append(arg_dict[id_list[3]])
    tmp_seq = " ".join(list(re.sub(r"[UZOB]", "X", str(seq_record.seq))))
    train_seq.append(tmp_seq)

for seq_record in SeqIO.parse("./data/test.fasta", "fasta"):
    # print(seq_record.id)
    test_name.append(seq_record.id)
    tmp_seq = " ".join(list(re.sub(r"[UZOB]", "X", str(seq_record.seq))))
    test_seq.append(tmp_seq)

for seq_record in SeqIO.parse("./data/val.fasta", "fasta"):
    if "sp" in seq_record.id:
        val_label.append(14)
    else:
        id_list = seq_record.id.split("|")
        val_label.append(arg_dict[id_list[3]])
    tmp_seq = " ".join(list(re.sub(r"[UZOB]", "X", str(seq_record.seq))))
    val_seq.append(tmp_seq)

print(len(train_seq))
print(len(label))
print(len(val_seq))
print(len(val_label))
print(len(test_seq))
print(len(test_name))

from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("facebook/esm2_t6_8M_UR50D")
train_encoding = tokenizer(train_seq, return_tensors="pt", max_length=2300, truncation=True, padding="max_length", verbose=True)
train_encoding, train_atten_mask_train = train_encoding["input_ids"], train_encoding["attention_mask"]







21209
21209
4387
4387
4469
4469


In [3]:
batch_size = 2
train_dataset = SeqDataset(train_encoding, torch.tensor(label), train_atten_mask_train)# torch.nn.functional.one_hot(torch.tensor(label)[:8], num_classes=15).to(torch.float))
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size,shuffle=True, num_workers=0)

# process the validation data and test data
val_encoding = tokenizer(val_seq, return_tensors="pt", max_length=2300, truncation=True, padding="max_length", verbose=True)
val_encoding, val_atten_mask_val = val_encoding["input_ids"], val_encoding["attention_mask"]
val_dataset = SeqDataset(val_encoding, torch.tensor(val_label), val_atten_mask_val)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size,shuffle=True, num_workers=0)

test_encoding = tokenizer(test_seq, return_tensors="pt", max_length=2300, truncation=True, padding="max_length", verbose=True)
test_encoding, test_atten_mask_test = test_encoding["input_ids"], test_encoding["attention_mask"]
test_dataset = SeqDataset(test_encoding, torch.zeros(len(test_seq)), test_atten_mask_test)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size,shuffle=False, num_workers=0)



In [5]:
# show the data in train_loader, val_loader and test_loader
for i in train_loader:
    print(i)
    break
for i in val_loader:
    print(i)
    break
for i in test_loader:
    print(i)
    break

{'labels': tensor([4, 5]), 'input_ids': tensor([[ 0, 20,  6,  ...,  1,  1,  1],
        [ 0, 20,  4,  ...,  1,  1,  1]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}
{'labels': tensor([14, 14]), 'input_ids': tensor([[ 0, 20,  5,  ...,  1,  1,  1],
        [ 0, 20,  7,  ...,  1,  1,  1]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}
{'labels': tensor([0., 0.]), 'input_ids': tensor([[ 0, 20, 19,  ...,  1,  1,  1],
        [ 0, 20,  4,  ...,  1,  1,  1]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}


In [5]:
train_dataset[0]

{'labels': tensor(2),
 'input_ids': tensor([ 0, 20,  8,  ...,  1,  1,  1]),
 'attention_mask': tensor([1, 1, 1,  ..., 0, 0, 0])}

In [6]:
from transformers import AutoTokenizer, EsmForSequenceClassification, AutoModelForSequenceClassification
from torch.optim import AdamW
from transformers import get_scheduler
import evaluate

model = AutoModelForSequenceClassification.from_pretrained("facebook/esm2_t6_8M_UR50D", num_labels=15)
model.to(device)
model.train()
epochs = 1

metric = evaluate.load("accuracy")

optimizer = AdamW(model.parameters(), lr=1e-5)
total_steps = len(train_loader) * epochs
scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=total_steps
)

from tqdm.auto import tqdm


progress_bar = tqdm(range(total_steps))
for epoch in range(epochs):
    for batch in train_loader:
        optimizer.zero_grad()
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        scheduler.step()


        progress_bar.update(1)
# save the model to this folder in pt
model.save_pretrained("./model")

model.eval()
for batch in tqdm(val_loader):
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    metric.add_batch(predictions=predictions, references=batch["labels"])

print(metric.compute())


Some weights of EsmForSequenceClassification were not initialized from the model checkpoint at facebook/esm2_t6_8M_UR50D and are newly initialized: ['classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight', 'classifier.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


  0%|          | 0/10605 [00:00<?, ?it/s]

  0%|          | 0/2194 [00:00<?, ?it/s]

{'accuracy': 0.9525871894232961}


In [7]:
model.eval()
for batch in tqdm(val_loader):
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    metric.add_batch(predictions=predictions, references=batch["labels"])

print(metric.compute())

  0%|          | 0/2194 [00:00<?, ?it/s]

{'accuracy': 0.9525871894232961}


In [8]:
model.eval()
for batch in tqdm(train_loader):
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    metric.add_batch(predictions=predictions, references=batch["labels"])

print(metric.compute())

  0%|          | 0/10605 [00:00<?, ?it/s]

{'accuracy': 0.9549719458720355}


In [9]:
test_encoding = tokenizer(test_seq, return_tensors="pt", max_length=2300, truncation=True, padding="max_length", verbose=True)
test_encoding, test_atten_mask_test = test_encoding["input_ids"], test_encoding["attention_mask"]
# int zero for the label
test_dataset = SeqDataset(test_encoding, torch.tensor([0]*len(test_seq)), test_atten_mask_test)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size,shuffle=False, num_workers=0)

In [10]:
# predeiction
prediction = []
model.eval()
for batch in tqdm(test_loader):
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    prediction.extend(predictions.cpu().numpy())

print(test_name,prediction)

  0%|          | 0/2235 [00:00<?, ?it/s]

['SEQ0', 'SEQ1', 'SEQ2', 'SEQ3', 'SEQ4', 'SEQ5', 'SEQ6', 'SEQ7', 'SEQ8', 'SEQ9', 'SEQ10', 'SEQ11', 'SEQ12', 'SEQ13', 'SEQ14', 'SEQ15', 'SEQ16', 'SEQ17', 'SEQ18', 'SEQ19', 'SEQ20', 'SEQ21', 'SEQ22', 'SEQ23', 'SEQ24', 'SEQ25', 'SEQ26', 'SEQ27', 'SEQ28', 'SEQ29', 'SEQ30', 'SEQ31', 'SEQ32', 'SEQ33', 'SEQ34', 'SEQ35', 'SEQ36', 'SEQ37', 'SEQ38', 'SEQ39', 'SEQ40', 'SEQ41', 'SEQ42', 'SEQ43', 'SEQ44', 'SEQ45', 'SEQ46', 'SEQ47', 'SEQ48', 'SEQ49', 'SEQ50', 'SEQ51', 'SEQ52', 'SEQ53', 'SEQ54', 'SEQ55', 'SEQ56', 'SEQ57', 'SEQ58', 'SEQ59', 'SEQ60', 'SEQ61', 'SEQ62', 'SEQ63', 'SEQ64', 'SEQ65', 'SEQ66', 'SEQ67', 'SEQ68', 'SEQ69', 'SEQ70', 'SEQ71', 'SEQ72', 'SEQ73', 'SEQ74', 'SEQ75', 'SEQ76', 'SEQ77', 'SEQ78', 'SEQ79', 'SEQ80', 'SEQ81', 'SEQ82', 'SEQ83', 'SEQ84', 'SEQ85', 'SEQ86', 'SEQ87', 'SEQ88', 'SEQ89', 'SEQ90', 'SEQ91', 'SEQ92', 'SEQ93', 'SEQ94', 'SEQ95', 'SEQ96', 'SEQ97', 'SEQ98', 'SEQ99', 'SEQ100', 'SEQ101', 'SEQ102', 'SEQ103', 'SEQ104', 'SEQ105', 'SEQ106', 'SEQ107', 'SEQ108', 'SEQ109', 'SEQ110',

In [11]:
print(prediction)

[0, 14, 11, 1, 2, 14, 11, 14, 0, 14, 14, 14, 14, 1, 7, 14, 14, 5, 14, 14, 14, 14, 1, 11, 5, 14, 5, 14, 14, 7, 14, 14, 1, 11, 11, 0, 13, 14, 14, 14, 14, 14, 14, 5, 5, 14, 14, 14, 14, 14, 14, 14, 11, 14, 14, 14, 14, 11, 2, 1, 14, 8, 14, 11, 2, 14, 14, 14, 14, 11, 11, 1, 14, 14, 14, 14, 14, 5, 11, 14, 11, 14, 14, 14, 7, 14, 5, 5, 5, 14, 14, 11, 11, 14, 14, 14, 14, 11, 11, 14, 2, 14, 14, 14, 14, 14, 11, 14, 14, 11, 14, 14, 5, 5, 14, 5, 6, 14, 5, 14, 14, 14, 11, 14, 14, 14, 14, 14, 5, 14, 14, 14, 0, 14, 14, 14, 7, 0, 1, 14, 14, 14, 14, 5, 14, 14, 14, 11, 11, 14, 6, 1, 1, 14, 14, 14, 14, 14, 14, 1, 14, 14, 14, 14, 14, 14, 14, 11, 14, 14, 14, 14, 11, 2, 14, 14, 14, 11, 14, 14, 1, 5, 14, 14, 14, 14, 2, 1, 5, 14, 11, 8, 14, 5, 14, 14, 14, 14, 14, 14, 14, 7, 14, 7, 0, 11, 14, 14, 11, 14, 14, 14, 14, 7, 7, 14, 14, 11, 14, 14, 14, 11, 14, 11, 14, 7, 14, 14, 7, 14, 14, 7, 14, 14, 5, 14, 14, 14, 1, 14, 14, 14, 11, 7, 14, 14, 14, 8, 14, 14, 14, 11, 14, 7, 8, 5, 14, 7, 11, 5, 11, 14, 14, 14, 7, 14, 14

In [13]:
for i in range(len(prediction)):
    prediction[i] += 10

In [14]:
print(len(test_name),len(prediction))
print(prediction)
# save 
# id,label
# SEQ0,10
# SEQ1,12
# SEQ2,9
# SEQ3,13
# SEQ4,12

with open("submission.csv", "w") as f:
    f.write("id,label\n")
    for i in range(len(test_name)):
        f.write(f"{test_name[i]},{prediction[i]}\n")

4469 4469
[10, 24, 21, 11, 12, 24, 21, 24, 10, 24, 24, 24, 24, 11, 17, 24, 24, 15, 24, 24, 24, 24, 11, 21, 15, 24, 15, 24, 24, 17, 24, 24, 11, 21, 21, 10, 23, 24, 24, 24, 24, 24, 24, 15, 15, 24, 24, 24, 24, 24, 24, 24, 21, 24, 24, 24, 24, 21, 12, 11, 24, 18, 24, 21, 12, 24, 24, 24, 24, 21, 21, 11, 24, 24, 24, 24, 24, 15, 21, 24, 21, 24, 24, 24, 17, 24, 15, 15, 15, 24, 24, 21, 21, 24, 24, 24, 24, 21, 21, 24, 12, 24, 24, 24, 24, 24, 21, 24, 24, 21, 24, 24, 15, 15, 24, 15, 16, 24, 15, 24, 24, 24, 21, 24, 24, 24, 24, 24, 15, 24, 24, 24, 10, 24, 24, 24, 17, 10, 11, 24, 24, 24, 24, 15, 24, 24, 24, 21, 21, 24, 16, 11, 11, 24, 24, 24, 24, 24, 24, 11, 24, 24, 24, 24, 24, 24, 24, 21, 24, 24, 24, 24, 21, 12, 24, 24, 24, 21, 24, 24, 11, 15, 24, 24, 24, 24, 12, 11, 15, 24, 21, 18, 24, 15, 24, 24, 24, 24, 24, 24, 24, 17, 24, 17, 10, 21, 24, 24, 21, 24, 24, 24, 24, 17, 17, 24, 24, 21, 24, 24, 24, 21, 24, 21, 24, 17, 24, 24, 17, 24, 24, 17, 24, 24, 15, 24, 24, 24, 11, 24, 24, 24, 21, 17, 24, 24, 24, 1

In [26]:
for batch in tqdm(test_loader):
    batch = {k: v.to(device) for k, v in batch.items()}
    print(batch)
    

  0%|          | 0/2235 [00:00<?, ?it/s]

{'labels': tensor([0, 0], device='cuda:0'), 'input_ids': tensor([[ 0, 20, 19,  ...,  1,  1,  1],
        [ 0, 20,  4,  ...,  1,  1,  1]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]], device='cuda:0')}
{'labels': tensor([0, 0], device='cuda:0'), 'input_ids': tensor([[ 0, 20, 10,  ...,  1,  1,  1],
        [ 0, 20, 16,  ...,  1,  1,  1]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]], device='cuda:0')}
{'labels': tensor([0, 0], device='cuda:0'), 'input_ids': tensor([[ 0, 20,  5,  ...,  1,  1,  1],
        [ 0, 20, 15,  ...,  1,  1,  1]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]], device='cuda:0')}
{'labels': tensor([0, 0], device='cuda:0'), 'input_ids': tensor([[ 0, 20, 20,  ...,  1,  1,  1],
        [ 0, 20, 21,  ...,  1,  1,  1]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [

KeyboardInterrupt: 

  0%|          | 0/2194 [00:00<?, ?it/s]

{'labels': tensor([11, 14], device='cuda:0'), 'input_ids': tensor([[ 0, 20,  7,  ...,  1,  1,  1],
        [ 0, 20, 12,  ...,  1,  1,  1]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]], device='cuda:0')}
{'labels': tensor([14,  2], device='cuda:0'), 'input_ids': tensor([[ 0, 20, 15,  ...,  1,  1,  1],
        [ 0, 20, 15,  ...,  1,  1,  1]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]], device='cuda:0')}
{'labels': tensor([14, 11], device='cuda:0'), 'input_ids': tensor([[ 0, 20,  4,  ...,  1,  1,  1],
        [ 0,  4, 12,  ...,  1,  1,  1]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]], device='cuda:0')}
{'labels': tensor([ 5, 11], device='cuda:0'), 'input_ids': tensor([[ 0, 20,  6,  ...,  1,  1,  1],
        [ 0, 20,  4,  ...,  1,  1,  1]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
 

KeyboardInterrupt: 